# Kapitola 12: BFS vs DFS - Dva zpusoby prohledavani

<div style="background: linear-gradient(90deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 10px; margin: 20px 0;">
    <h2 style="color: white; margin: 0;">Blok 2 | Algoritmy a hledani</h2>
    <p style="color: white; margin: 10px 0;">Porovnani prohledavacich strategii</p>
</div>

## Co se naucite v teto kapitole

1. **BFS (Breadth-First Search)** - Prohledavani do sirky
2. **DFS (Depth-First Search)** - Prohledavani do hloubky
3. **Fronta vs Zasobnik** - Klicovy rozdil v datovych strukturach
4. **Vizualizace** - Jak oba algoritmy prozkoumavaji graf
5. **Kdy pouzit ktery** - Vyber spravneho nastroje

---

In [ ]:
# Inicializace prostredi pro Google Colab
import sys
from datetime import datetime

print("=" * 60)
print("KAPITOLA 12: BFS vs DFS")
print("=" * 60)
print(f"Python verze: {sys.version.split()[0]}")
print(f"Datum: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print("=" * 60)

# Instalace knihoven
!pip install -q networkx matplotlib

print("\nProstredi pripraveno!")

---

## Motivace: Dva zpusoby hledani v bludisti

Predstavte si bludiste. Jak ho budete prohledavat?

### Strategie 1: Opatrny pruzkumnik (BFS)
- Prozkouma vsechny chodby na dosah
- Pak vsechny o krok dal
- Postupuje **vrstvu po vrstve**
- Jako vlna na vode

### Strategie 2: Odvazny dobrodruh (DFS)
- Vybere jednu chodbu a jde az na konec
- Kdyz narazi na slepu ulicku, vrati se
- Zkusi jinou cestu
- Vrha se **do hloubky**

In [ ]:
# Nase bludiste jako graf

bludiste = {
    'A': ['B', 'C'],
    'B': ['A', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['B'],
    'E': ['B', 'F'],
    'F': ['C', 'E', 'G'],
    'G': ['F']  # Cil - poklad!
}

start = 'A'
cil = 'G'

print("NASE BLUDISTE")
print("=" * 40)
print()
print("       A")
print("      / \\")
print("     B   C")
print("    /|   |")
print("   D E---F")
print("         |")
print("         G (CIL)")
print()
print("Reprezentace jako slovnik:")
for uzel, sousede in bludiste.items():
    print(f"   {uzel} -> {sousede}")

---

## Klicovy rozdil: Fronta vs Zasobnik

### Fronta (Queue) - FIFO
- **F**irst **I**n, **F**irst **O**ut
- Jako fronta v obchode: kdo prijde prvni, odchazi prvni
- Pouziva **BFS**

### Zasobnik (Stack) - LIFO
- **L**ast **I**n, **F**irst **O**ut  
- Jako kominek taliru: posledni polozeny beres prvni
- Pouziva **DFS**

In [ ]:
from collections import deque

# Demonstrace fronty vs zasobniku

print("FRONTA (Queue) - FIFO")
print("=" * 40)
fronta = deque()
print("Vkladame: A, B, C")
fronta.append('A')
fronta.append('B')
fronta.append('C')
print(f"Fronta: {list(fronta)}")
print(f"Vybirame: {fronta.popleft()}")
print(f"Vybirame: {fronta.popleft()}")
print(f"Vybirame: {fronta.popleft()}")
print("-> Poradi: A, B, C (jako vkladani)")

print()
print("ZASOBNIK (Stack) - LIFO")
print("=" * 40)
zasobnik = []
print("Vkladame: A, B, C")
zasobnik.append('A')
zasobnik.append('B')
zasobnik.append('C')
print(f"Zasobnik: {zasobnik}")
print(f"Vybirame: {zasobnik.pop()}")
print(f"Vybirame: {zasobnik.pop()}")
print(f"Vybirame: {zasobnik.pop()}")
print("-> Poradi: C, B, A (obracene!)")

---

## BFS: Prohledavani do sirky

**Algoritmus:**
1. Vloz pocatecni uzel do fronty
2. Dokud neni fronta prazdna:
   - Vyber prvni uzel z fronty
   - Pokud je to cil, konec!
   - Pridej vsechny nenavstivene sousedy **na konec** fronty

In [ ]:
from collections import deque

def bfs(graf, start, cil):
    """
    Prohledavani do sirky (Breadth-First Search).
    Najde nejkratsi cestu.
    """
    # Fronta: [(uzel, cesta_k_nemu)]
    fronta = deque([(start, [start])])
    navstivene = {start}
    
    kroky = []  # Pro vizualizaci
    
    while fronta:
        uzel, cesta = fronta.popleft()  # Vyber z ZACATKU
        kroky.append((uzel, list(fronta)))
        
        if uzel == cil:
            return cesta, kroky
        
        for soused in graf.get(uzel, []):
            if soused not in navstivene:
                navstivene.add(soused)
                fronta.append((soused, cesta + [soused]))
    
    return None, kroky

# Spusteni BFS
print("BFS - PROHLEDAVANI DO SIRKY")
print("=" * 50)

cesta_bfs, kroky_bfs = bfs(bludiste, start, cil)

print("\nPostup prohledavani (uzel -> fronta):")
for i, (uzel, fronta_stav) in enumerate(kroky_bfs):
    fronta_str = [u for u, _ in fronta_stav] if fronta_stav else []
    print(f"   {i+1}. Zpracovavam: {uzel}, Fronta: {fronta_str}")

print(f"\nNalezena cesta: {' -> '.join(cesta_bfs)}")
print(f"Delka: {len(cesta_bfs)-1} kroku")

---

## DFS: Prohledavani do hloubky

**Algoritmus:**
1. Vloz pocatecni uzel na zasobnik
2. Dokud neni zasobnik prazdny:
   - Vyber uzel z vrcholu zasobniku
   - Pokud je to cil, konec!
   - Pridej vsechny nenavstivene sousedy **na vrchol** zasobniku

In [ ]:
def dfs(graf, start, cil):
    """
    Prohledavani do hloubky (Depth-First Search).
    Najde nejakou cestu, ne nutne nejkratsi.
    """
    # Zasobnik: [(uzel, cesta_k_nemu)]
    zasobnik = [(start, [start])]
    navstivene = set()
    
    kroky = []  # Pro vizualizaci
    
    while zasobnik:
        uzel, cesta = zasobnik.pop()  # Vyber z KONCE (vrcholu)
        
        if uzel in navstivene:
            continue
        
        navstivene.add(uzel)
        kroky.append((uzel, list(zasobnik)))
        
        if uzel == cil:
            return cesta, kroky
        
        # reversed() pro intuitivnejsi poradi
        for soused in reversed(graf.get(uzel, [])):
            if soused not in navstivene:
                zasobnik.append((soused, cesta + [soused]))
    
    return None, kroky

# Spusteni DFS
print("DFS - PROHLEDAVANI DO HLOUBKY")
print("=" * 50)

cesta_dfs, kroky_dfs = dfs(bludiste, start, cil)

print("\nPostup prohledavani (uzel -> zasobnik):")
for i, (uzel, zasobnik_stav) in enumerate(kroky_dfs):
    zasobnik_str = [u for u, _ in zasobnik_stav] if zasobnik_stav else []
    print(f"   {i+1}. Zpracovavam: {uzel}, Zasobnik: {zasobnik_str}")

print(f"\nNalezena cesta: {' -> '.join(cesta_dfs)}")
print(f"Delka: {len(cesta_dfs)-1} kroku")

---

## Porovnani BFS vs DFS

In [ ]:
# Porovnani vysledku

print("POROVNANI BFS vs DFS")
print("=" * 50)

print(f"\nBFS cesta: {' -> '.join(cesta_bfs)} (delka: {len(cesta_bfs)-1})")
print(f"DFS cesta: {' -> '.join(cesta_dfs)} (delka: {len(cesta_dfs)-1})")

print(f"\nBFS prozkoumanych uzlu: {len(kroky_bfs)}")
print(f"DFS prozkoumanych uzlu: {len(kroky_dfs)}")

print("\n" + "-" * 50)
print("\nKlicove rozdily:")
print("""
   BFS (Sirka)              |  DFS (Hloubka)
   -------------------------|-------------------------
   Pouziva FRONTU (FIFO)    |  Pouziva ZASOBNIK (LIFO)
   Najde NEJKRATSI cestu    |  Najde NEJAKOU cestu
   Vyssi pamet (vsechny     |  Nizsi pamet (jen aktualni
   uzly ve vrstve)          |  cesta)
   Dobry pro nejblizsi cil  |  Dobry pro existenci reseni
""")

---

## Vizualizace postupu prohledavani

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

def vizualizuj_prohledavani(graf, kroky, cesta, titulek):
    """
    Vizualizuje postup prohledavani.
    """
    # Vytvor networkx graf
    G = nx.Graph(graf)
    pos = {
        'A': (1, 3),
        'B': (0, 2),
        'C': (2, 2),
        'D': (0, 1),
        'E': (1, 1),
        'F': (2, 1),
        'G': (2, 0)
    }
    
    # Poradi navstiveni
    poradi = [uzel for uzel, _ in kroky]
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Graf 1: Poradi navstiveni
    ax1 = axes[0]
    
    node_colors = []
    for node in G.nodes():
        if node == 'A':
            node_colors.append('#2ecc71')  # Start - zelena
        elif node == 'G':
            node_colors.append('#e74c3c')  # Cil - cervena
        elif node in poradi:
            node_colors.append('#3498db')  # Navstivene - modra
        else:
            node_colors.append('#bdc3c7')  # Nenavstivene - seda
    
    nx.draw(G, pos, ax=ax1, with_labels=True, node_size=1500, 
            node_color=node_colors, font_size=14, font_weight='bold')
    
    # Pridej cisla poradi
    for i, uzel in enumerate(poradi, 1):
        x, y = pos[uzel]
        ax1.annotate(str(i), (x, y-0.3), fontsize=12, ha='center', 
                    color='white', fontweight='bold',
                    bbox=dict(boxstyle='circle', facecolor='#9b59b6'))
    
    ax1.set_title(f"{titulek}\nPoradi navstiveni (cisla)", fontsize=12)
    
    # Graf 2: Nalezena cesta
    ax2 = axes[1]
    
    node_colors2 = []
    for node in G.nodes():
        if node in cesta:
            node_colors2.append('#f39c12')  # Na ceste - oranzova
        else:
            node_colors2.append('#bdc3c7')  # Mimo cestu - seda
    
    nx.draw(G, pos, ax=ax2, with_labels=True, node_size=1500,
            node_color=node_colors2, font_size=14, font_weight='bold')
    
    # Zvyrazni hrany cesty
    path_edges = [(cesta[i], cesta[i+1]) for i in range(len(cesta)-1)]
    nx.draw_networkx_edges(G, pos, ax=ax2, edgelist=path_edges,
                          edge_color='#e74c3c', width=3)
    
    ax2.set_title(f"Nalezena cesta\n{' -> '.join(cesta)}", fontsize=12)
    
    plt.tight_layout()
    plt.show()

# Vizualizace BFS
print("VIZUALIZACE BFS")
vizualizuj_prohledavani(bludiste, kroky_bfs, cesta_bfs, "BFS - Prohledavani do sirky")

# Vizualizace DFS
print("\nVIZUALIZACE DFS")
vizualizuj_prohledavani(bludiste, kroky_dfs, cesta_dfs, "DFS - Prohledavani do hloubky")

---

## Animace prohledavani

In [ ]:
def animuj_textove(graf, kroky, nazev):
    """
    Textova animace prohledavani.
    """
    print(f"\n{nazev}")
    print("=" * 50)
    
    vsechny_uzly = list(graf.keys())
    navstivene = set()
    
    for i, (uzel, _) in enumerate(kroky, 1):
        navstivene.add(uzel)
        
        # Zobraz stav
        stav = ""
        for u in vsechny_uzly:
            if u == uzel:
                stav += f"[{u}]"
            elif u in navstivene:
                stav += f" {u} "
            else:
                stav += " . "
        
        print(f"Krok {i}: {stav}  <- Zpracovavam {uzel}")
    
    print("\n[X] = aktualni uzel")
    print(" X  = jiz navstiveny")
    print(" .  = nenavstiveny")

# Animace
animuj_textove(bludiste, kroky_bfs, "BFS ANIMACE")
animuj_textove(bludiste, kroky_dfs, "DFS ANIMACE")

---

## Kdy pouzit BFS vs DFS?

### Pouzijte BFS kdyz:
- Potrebujete **nejkratsi cestu**
- Hledate neco **blizko startu**
- Graf neni prilis velky (pamet)
- Priklady: nejblizsi obchod, nejkratsi cesta na mape

### Pouzijte DFS kdyz:
- Staci **jakekoli reseni**
- Graf je velmi hluboky
- Chcete prozkoumat **vsechny moznosti**
- Priklady: reseni hlavolamu, hledani v souborovem systemu

In [ ]:
# Priklad: Hledani vsech cest (DFS je lepsi)

def najdi_vsechny_cesty(graf, start, cil, cesta=None):
    """
    Najde vsechny mozne cesty pomoci DFS (rekurze).
    """
    if cesta is None:
        cesta = [start]
    
    if start == cil:
        return [cesta]
    
    vsechny_cesty = []
    
    for soused in graf.get(start, []):
        if soused not in cesta:  # Zabranime cyklum
            nove_cesty = najdi_vsechny_cesty(graf, soused, cil, cesta + [soused])
            vsechny_cesty.extend(nove_cesty)
    
    return vsechny_cesty

# Najdi vsechny cesty z A do G
print("VSECHNY MOZNE CESTY z A do G")
print("=" * 50)

vsechny = najdi_vsechny_cesty(bludiste, 'A', 'G')

for i, cesta in enumerate(vsechny, 1):
    print(f"   {i}. {' -> '.join(cesta)} (delka: {len(cesta)-1})")

print(f"\nCelkem {len(vsechny)} ruznych cest.")
print("\nNejkratsi cesta (kterou by nasel BFS):")
nejkratsi = min(vsechny, key=len)
print(f"   {' -> '.join(nejkratsi)} (delka: {len(nejkratsi)-1})")

---

## Cviceni

In [ ]:
# CVICENI 1: Vlastni bludiste
# =============================
# Vytvorte vlastni bludiste a porovnejte BFS a DFS

print("CVICENI 1: Vlastni bludiste")
print("=" * 50)

# Vase bludiste (upravte):
moje_bludiste = {
    'START': ['A', 'B'],
    'A': ['START', 'C', 'D'],
    'B': ['START', 'E'],
    'C': ['A'],
    'D': ['A', 'F'],
    'E': ['B', 'F'],
    'F': ['D', 'E', 'CIL'],
    'CIL': ['F']
}

# Test
cesta_b, _ = bfs(moje_bludiste, 'START', 'CIL')
cesta_d, _ = dfs(moje_bludiste, 'START', 'CIL')

print(f"BFS: {' -> '.join(cesta_b)} (delka: {len(cesta_b)-1})")
print(f"DFS: {' -> '.join(cesta_d)} (delka: {len(cesta_d)-1})")

In [ ]:
# CVICENI 2: Mrizka (grid)
# =========================
# Vytvorte BFS pro 2D mrizku

print("CVICENI 2: Hledani cesty v mrizce")
print("=" * 50)

def vytvor_mrizku(rows, cols):
    """Vytvori graf reprezentujici mrizku."""
    graf = {}
    for r in range(rows):
        for c in range(cols):
            uzel = (r, c)
            sousede = []
            # Nahoru, dolu, vlevo, vpravo
            for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                nr, nc = r + dr, c + dc
                if 0 <= nr < rows and 0 <= nc < cols:
                    sousede.append((nr, nc))
            graf[uzel] = sousede
    return graf

def bfs_mrizka(graf, start, cil):
    """BFS pro mrizku."""
    fronta = deque([(start, [start])])
    navstivene = {start}
    
    while fronta:
        uzel, cesta = fronta.popleft()
        if uzel == cil:
            return cesta
        for soused in graf.get(uzel, []):
            if soused not in navstivene:
                navstivene.add(soused)
                fronta.append((soused, cesta + [soused]))
    return None

# Test na 4x4 mrizce
mrizka = vytvor_mrizku(4, 4)
start_m = (0, 0)  # Levy horni roh
cil_m = (3, 3)    # Pravy dolni roh

cesta_m = bfs_mrizka(mrizka, start_m, cil_m)

print(f"\nMrizka 4x4")
print(f"Start: {start_m}, Cil: {cil_m}")
print(f"Nejkratsi cesta ({len(cesta_m)-1} kroku):")

# Vizualizace v mrizce
for r in range(4):
    radek = ""
    for c in range(4):
        if (r, c) == start_m:
            radek += "[S]"
        elif (r, c) == cil_m:
            radek += "[C]"
        elif (r, c) in cesta_m:
            radek += " * "
        else:
            radek += " . "
    print(f"   {radek}")

In [ ]:
# CVICENI 3: Pocet navstivenych uzlu
# ===================================
# Porovnejte efektivitu BFS a DFS na ruznych grafech

print("CVICENI 3: Efektivita BFS vs DFS")
print("=" * 50)

# Ruzne grafy
grafy = {
    "Linearni": {
        'A': ['B'], 'B': ['A', 'C'], 'C': ['B', 'D'], 
        'D': ['C', 'E'], 'E': ['D']
    },
    "Strom": {
        'A': ['B', 'C'], 
        'B': ['A', 'D', 'E'], 
        'C': ['A', 'F', 'G'],
        'D': ['B'], 'E': ['B'], 'F': ['C'], 'G': ['C']
    },
    "Huste propojeny": {
        'A': ['B', 'C', 'D'],
        'B': ['A', 'C', 'D', 'E'],
        'C': ['A', 'B', 'D', 'E'],
        'D': ['A', 'B', 'C', 'E'],
        'E': ['B', 'C', 'D']
    }
}

print("\n          Graf        | BFS kroky | DFS kroky | Rozdil")
print("-" * 60)

for nazev, graf in grafy.items():
    uzly = list(graf.keys())
    start_g = uzly[0]
    cil_g = uzly[-1]
    
    _, kroky_b = bfs(graf, start_g, cil_g)
    _, kroky_d = dfs(graf, start_g, cil_g)
    
    rozdil = len(kroky_d) - len(kroky_b)
    symbol = "+" if rozdil > 0 else ""
    
    print(f"{nazev:20} |    {len(kroky_b):2}     |    {len(kroky_d):2}     |  {symbol}{rozdil}")

---

## Rekurzivni DFS

DFS lze implementovat i rekurzivne (elegantnejsi, ale omezeno velikosti zasobniku):

In [ ]:
def dfs_rekurzivni(graf, start, cil, navstivene=None, cesta=None):
    """
    Rekurzivni implementace DFS.
    """
    if navstivene is None:
        navstivene = set()
    if cesta is None:
        cesta = []
    
    navstivene.add(start)
    cesta = cesta + [start]
    
    if start == cil:
        return cesta
    
    for soused in graf.get(start, []):
        if soused not in navstivene:
            vysledek = dfs_rekurzivni(graf, soused, cil, navstivene, cesta)
            if vysledek:
                return vysledek
    
    return None

# Test
print("REKURZIVNI DFS")
print("=" * 50)

cesta_rek = dfs_rekurzivni(bludiste, 'A', 'G')
print(f"Nalezena cesta: {' -> '.join(cesta_rek)}")

---

## Shrnuti kapitoly

<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 30px; border-radius: 10px; color: white; margin: 30px 0;">
    <h3 style="color: white; margin-top: 0;">Co jste se naucili</h3>
    <ul style="list-style: none; padding-left: 0;">
        <li>* BFS = prohledavani do SIRKY pomoci FRONTY (FIFO)</li>
        <li>* DFS = prohledavani do HLOUBKY pomoci ZASOBNIKU (LIFO)</li>
        <li>* BFS najde NEJKRATSI cestu</li>
        <li>* DFS najde NEJAKOU cestu (rychleji pro hluboke grafy)</li>
        <li>* Volba algoritmu zavisi na typu problemu</li>
    </ul>
    
    <h3 style="color: white;">Klicove dovednosti</h3>
    <p>Umite implementovat BFS i DFS a vybrat spravny algoritmus pro dany problem.</p>
    
    <h3 style="color: white;">Dalsi kapitola</h3>
    <p>Kapitola 13: Dijkstruv algoritmus - hledani nejkratsi cesty s vahami</p>
</div>

---

*Notebook pro kurz AI - Blok 2: Algoritmy a hledani*  
*Kompatibilni s Google Colab*